In [ ]:
import os, json
import openai

from dotenv import load_dotenv
load_dotenv(".env")

aoai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
aoai_api_key = os.environ["AZURE_OPENAI_API_KEY"]
api_version = os.environ["AZURE_OPENAI_API_VERSION"]
chat_model = os.environ["AZURE_OPENAI_CHAT_MODEL"]

client = openai.AzureOpenAI( 
    azure_endpoint=aoai_endpoint,
    api_key=aoai_api_key,
    api_version= api_version
)

In [ ]:
import base64
from mimetypes import guess_type

# Function to encode a local image into data URL 
def local_image_to_data_url(image_path):
    # Guess the MIME type of the image based on the file extension
    mime_type, _ = guess_type(image_path)
    if mime_type is None:
        mime_type = 'application/octet-stream'  # Default MIME type if none is found

    # Read and encode the image file
    with open(image_path, "rb") as image_file:
        base64_encoded_data = base64.b64encode(image_file.read()).decode('utf-8')

    # Construct the data URL
    return base64_encoded_data


def call_o1_model(messages):
    response = client.chat.completions.create(
        model=chat_model, 
        messages=messages,
        reasoning_effort="high",
    #    max_completion_tokens=4000 
    )
    response_message = response.choices[0].message
    messages.append(response_message)
    answer = response_message.content
    return answer, messages


In [ ]:
base64_sample = local_image_to_data_url("./sample3.png")
base64_conn1 = local_image_to_data_url("./conn03-0.png")
url_sample = {"url": "data:image/jpeg;base64," + base64_sample}
url_conn1 = {"url": "data:image/jpeg;base64," + base64_conn1}

In [ ]:
with open("./user_promp_template.txt", "r", encoding = 'utf-8') as f:
    user_prompt_template = f.read()

messages = []
content = []
content.append({"type": "text", "text": user_prompt_template})
content.append({"type": "text", "text":"attachment 1"})
content.append({"type": "image_url", "image_url": url_sample})
content.append({"type": "text", "text":"attachment 2"})
content.append({"type": "image_url", "image_url": url_conn1})

content.append({"type": "text", "text": "\
                <Placeholder for the subsequent questions>\
                "})

messages.append({"role": "user","content":content})

In [ ]:
answer, messages = call_o1_model(messages)
print(answer)

In [ ]:
content = []
content.append({"type": "text", "text": "\
                <Placeholder for the subsequent questions>\
                "})
messages.append({"role": "user","content": content})

answer, messages = call_o1_model(messages)
print(answer)